<a href="https://colab.research.google.com/github/emmacionca/data_curation_and_analysis/blob/main/eac4697_data_curation_and_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot as plt
print ("All the required libraries are imported.")

All the required libraries are imported.


Import libraries to parse web-data through web-scraping APIs.